[View in Colaboratory](https://colab.research.google.com/github/eurismarpires/train_music21/blob/master/Gan_teste.ipynb)

# **Preparação do Ambiente**

http://www.rricard.me/machine/learning/generative/adversarial/networks/keras/tensorflow/2017/04/05/gans-part2.html

In [1]:
!ls

datalab


In [2]:
!wget https://www.dropbox.com/s/jbo9n4vz8u7m9nf/Pop_Music_Midi.zip?dl=1

--2018-07-07 14:14:30--  https://www.dropbox.com/s/jbo9n4vz8u7m9nf/Pop_Music_Midi.zip?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.7.1, 2620:100:601f:1::a27d:901
Connecting to www.dropbox.com (www.dropbox.com)|162.125.7.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/jbo9n4vz8u7m9nf/Pop_Music_Midi.zip [following]
--2018-07-07 14:14:30--  https://www.dropbox.com/s/dl/jbo9n4vz8u7m9nf/Pop_Music_Midi.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc03c041a42062e6504119390c90.dl.dropboxusercontent.com/cd/0/get/AKwCgNaa766jV3BXObIYR0QqyK1YB3c4a4nH9ACVMDLfrw12yrbnai47OvOtS3dBPSQCuO_EoyYLjX1AYB-T422gr0Vji00TLbNB9FeZlK1hYAnmA3gUUbSOo1GlgSBu7WyFSIoj4iPPGXnLqLsaTKv0aLPchMVxzU_A-yS9plozpBB8PqFxWPGUnZCMixTw-Lw/file?dl=1 [following]
--2018-07-07 14:14:31--  https://uc03c041a42062e6504119390c90.dl.dropboxusercontent.com/cd/0/get/AKwCgNaa766jV3BXObIYR0QqyK1YB3c

In [3]:
!unzip Pop_Music_Midi.zip?dl=1

Archive:  Pop_Music_Midi.zip?dl=1
   creating: Pop_Music_Midi/
  inflating: Pop_Music_Midi/All The Small Things - Chorus.midi  
  inflating: Pop_Music_Midi/All The Small Things - Verse.midi  
  inflating: Pop_Music_Midi/Around The World - Chorus.midi  
  inflating: Pop_Music_Midi/Around The World - Verse.midi  
  inflating: Pop_Music_Midi/Baby I'm Yours - Chorus.midi  
  inflating: Pop_Music_Midi/Baby One More Time - Chorus.midi  
  inflating: Pop_Music_Midi/Baby One More Time - Verse.midi  
  inflating: Pop_Music_Midi/Barbie Girl - Chorus.midi  
  inflating: Pop_Music_Midi/Barbie Girl - Verse.midi  
  inflating: Pop_Music_Midi/Beautiful Life - Chorus.midi  
  inflating: Pop_Music_Midi/Beautiful Life - Pre Chorus.midi  
  inflating: Pop_Music_Midi/Beautiful Life - Verse.midi  
  inflating: Pop_Music_Midi/Best Day Of My Life - Chorus.midi  
  inflating: Pop_Music_Midi/Blank Space - Chorus.midi  
  inflating: Pop_Music_Midi/Blank Space - Pre Chorus.midi  
  inflating: Pop_Music_Midi/Blan

In [0]:
!rm Pop_Music_Midi.zip?dl=1

In [5]:
!ls

datalab  Pop_Music_Midi


In [6]:
!pip install tqdm

    100% |████████████████████████████████| 51kB 2.1MB/s 


In [7]:
!pip install music21

    100% |████████████████████████████████| 17.9MB 2.3MB/s 
  Running setup.py bdist_wheel for music21 ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / done
  Stored in directory: /content/.cache/pip/wheels/66/0d/71/ed677f5bec1eca6f924423a317eeb7e4df4859353861d58d44
Successfully built music21


# **Aqui começa o código de fato**

In [8]:
%matplotlib inline
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm_notebook as tqdm
from keras.models import Model
from keras.layers import Input, Reshape, TimeDistributed
from keras.layers.core import Dense, Activation, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import UpSampling1D, Conv1D
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam, SGD
from keras.callbacks import TensorBoard
from music21 import *
import glob
import numpy as np
from keras.models import Sequential, load_model
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.callbacks import History, ModelCheckpoint
from keras.optimizers import RMSprop
from datetime import datetime

Using TensorFlow backend.


In [0]:
path = 'Pop_Music_Midi/'

In [0]:
files = glob.glob('{}/*.mid*'.format(path))

In [0]:
KEY_SIG_OFFSET = 0 
melody_corpus = []
for i, file in enumerate(files[:2]):
#for file in files:
    midi_stream = converter.parse(file)
    
    part0 = midi_stream[0]
    key_sig = part0.keySignature
    midi_stream.transpose(KEY_SIG_OFFSET - key_sig.tonic.pitchClass, inPlace=True)
    for m in part0.measures(1, None):
        if hasattr(m, 'flat'):
            for nr in m.flat.notesAndRests:
                pitch = nr.pitch.midi  if isinstance(nr, note.Note) else 0
                duration = float(nr.quarterLength)
                note_repr = (pitch, duration)
                melody_corpus.append(note_repr)

In [0]:
melody_set = set(melody_corpus)

In [0]:
notes_indices = {note: i for i, note in enumerate(melody_set)}
indices_notes = {i: note for i, note in enumerate(melody_set)}

In [14]:
corpus_size = len(melody_set)
print(corpus_size)

18


In [0]:
phrase_len = 20
step_size = 3

In [0]:
phrases = []
next_notes = []

In [17]:
for i in range(0, len(melody_corpus) - phrase_len, step_size):
    phrases.append(melody_corpus[i: i + phrase_len])
    next_notes.append(melody_corpus[i + phrase_len])
print('nb sequences:', len(phrases))

nb sequences: 25


In [18]:
# transform data into binary matrices
X = np.zeros((len(phrases), phrase_len, corpus_size), dtype=np.bool)
y = np.zeros((len(phrases), corpus_size), dtype=np.bool)
print(X.shape)
print(y.shape)

(25, 20, 18)
(25, 18)


In [0]:
for i, phrase in enumerate(phrases):
    for j, note in enumerate(phrase):
        X[i, j, notes_indices[note]] = 1
    y[i, notes_indices[next_notes[i]]] = 1

In [0]:
layer_size=128
batch_size = 128
nb_epoch = 400

In [21]:
corpus_size

18

In [22]:
phrase_len

20

In [23]:
def get_gen(G_in, phrase_len=phrase_len, corpus_size=corpus_size):
    #G_in = Input(shape=(phrase_len, corpus_size))
    layer = LSTM(layer_size, return_sequences=True)(G_in)
    layer = Dropout(0.2)(layer)
    layer = LSTM(layer_size, return_sequences=True)(layer)
    layer = Dropout(0.2)(layer)
    layer = TimeDistributed(Dense(corpus_size))(layer)
    G_out = Activation('softmax')(layer)
    LSTM_out = Activation('softmax')(layer)
    G = Model(G_in, G_out)
    G.compile(loss='categorical_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])
    return G, G_out
G_in = Input(shape=(phrase_len, corpus_size))
G, G_out = get_gen(G_in)
G.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 20, 18)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 20, 128)           75264     
_________________________________________________________________
dropout_1 (Dropout)          (None, 20, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 20, 128)           131584    
_________________________________________________________________
dropout_2 (Dropout)          (None, 20, 128)           0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 20, 18)            2322      
_________________________________________________________________
activation_1 (Activation)    (None, 20, 18)            0         
Total para

In [0]:
def get_dis(D_in, phrase_len=phrase_len, corpus_size=corpus_size):
    
    #G_in = Input(shape=(phrase_len, corpus_size))
    layer = LSTM(layer_size, return_sequences=True)(D_in)
    layer = Dropout(0.2)(layer)
    layer = LSTM(layer_size)(layer)
    layer = Dropout(0.2)(layer)
    layer = Dense(2)(layer)
    D_out = Activation('sigmoid')(layer)
    D = Model(D_in, D_out)
    D.compile(loss='binary_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])
    return D, D_out

In [0]:
D_in = Input(shape=(phrase_len, corpus_size))

In [26]:
D, D_out = get_dis(D_in)
D.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 20, 18)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 20, 128)           75264     
_________________________________________________________________
dropout_3 (Dropout)          (None, 20, 128)           0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 258       
_________________________________________________________________
activation_3 (Activation)    (None, 2)                 0         
Total para

In [0]:
def set_trainability(model, trainable=False):
    model.trainable = trainable
    for layer in model.layers:
        layer.trainable = trainable

In [0]:
def make_gan(GAN_in, G, D):
    set_trainability(D, False)
    x = G(GAN_in) # ENTRADA DA GAN
    GAN_out = D(x)
    GAN = Model(GAN_in, GAN_out)
    GAN.compile(loss='binary_crossentropy', optimizer=G.optimizer)
    return GAN, GAN_out

In [29]:
GAN_in = Input(shape=(phrase_len, corpus_size))
GAN, GAN_out = make_gan(GAN_in, G, D)
GAN.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 20, 18)            0         
_________________________________________________________________
model_1 (Model)              (None, 20, 18)            209170    
_________________________________________________________________
model_2 (Model)              (None, 2)                 207106    
Total params: 416,276
Trainable params: 209,170
Non-trainable params: 207,106
_________________________________________________________________


In [30]:
X.shape

(25, 20, 18)

In [0]:
n_samples = 10
noise_dim = X.shape[2]

In [0]:
def sample_data_and_gen(G,n_samples=n_samples,noise_dim=noise_dim):
    XT = X[:n_samples]
    XN_noise = np.random.uniform(0, 1, size=[n_samples,phrase_len,noise_dim])
    XN = G.predict(XN_noise)
    print(XT.shape,XN.shape)
    XX = np.concatenate((XT, XN))
    yy = np.zeros((2*n_samples, 2))
    yy[:n_samples, 1] = 1
    yy[n_samples:, 0] = 1
    return XX, yy

In [0]:
def pretrain(G, D, noise_dim=10, n_samples=10000, batch_size=32):
    X, y = sample_data_and_gen(G, n_samples=n_samples, noise_dim=noise_dim)
    set_trainability(D, True)
    D.fit(X, y, epochs=1, batch_size=batch_size)

In [34]:
pretrain(G, D,noise_dim = noise_dim, n_samples=n_samples,batch_size=1)

(10, 20, 18) (10, 20, 18)
Epoch 1/1
20/20 [==============================] - 3s 148ms/step - loss: 0.5643 - acc: 0.7750


In [0]:
def sample_noise(G, noise_dim=10, n_samples=10000):
    X = np.random.uniform(0, 1, size=[n_samples,phrase_len, noise_dim])
    y = np.zeros((n_samples, 2))
    y[:, 1] = 1
    return X, y

In [0]:
d_loss = []
g_loss = []

In [37]:
d_loss = []
for epoch in range(2):
    X, y = sample_data_and_gen(G, n_samples=10, noise_dim=noise_dim)
    set_trainability(D, True)
    d_loss.append(D.train_on_batch(X, y))
    X, y = sample_noise(G, n_samples=n_samples, noise_dim=noise_dim)
    set_trainability(D, False)
    g_loss.append(GAN.train_on_batch(X, y))
    print("Epoch #{}: Generative Loss: {}, Discriminative Loss: {}".format(epoch + 1, g_loss[-1], d_loss[-1]))

(10, 20, 18) (10, 20, 18)
Epoch #1: Generative Loss: 3.8601810932159424, Discriminative Loss: [0.08657917, 1.0]
(10, 20, 18) (10, 20, 18)
Epoch #2: Generative Loss: 2.866421937942505, Discriminative Loss: [2.1913166, 0.5]
